In [ ]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import units as u
from astropy.table import Table
from astropy.coordinates import SkyCoord, Angle

import astroquery
from astroquery.sdss import SDSS

%matplotlib inline
print astroquery.__version__
print pd.__version__

### Primero analizamos las galaxias elipticas de GalaxyZOO

In [ ]:
cat_url='https://raw.githubusercontent.com/rpmunoz/PentaUC/master/Astronomia%20era%20informacion/data/SDSS_galaxyzoo_elipticas.csv'
cat_galaxyzoo=pd.read_csv(cat_url,header=0,sep=',')
cat_galaxyzoo.head()

In [ ]:
coo=SkyCoord(cat_galaxyzoo['ra'], cat_galaxyzoo['dec'], unit=(u.deg,u.deg))
for i in range(len(coo)):
    print 'Processing galaxy ', i, ' - Coo: ', coo[i]
    result = SDSS.query_region(coo[i], spectro=True, photoobj_fields=['ra','dec','u','g','r','i','z'], radius=2*u.arcsec)
    if i==0:
        sdss_data=result
    else:
        sdss_data.add_row(np.array(result)[0])

cat_sdss_elipticas=pd.DataFrame(np.array(sdss_data))
cat_sdss_elipticas

### Luego analizamos las galaxias espirales de GalaxyZOO

In [ ]:
cat_url='https://raw.githubusercontent.com/rpmunoz/PentaUC/master/Astronomia%20era%20informacion/data/SDSS_galaxyzoo_espirales.csv'
cat_galaxyzoo=pd.read_csv(cat_url,header=0,sep=',')
cat_galaxyzoo.head()

In [ ]:
coo=SkyCoord(cat_galaxyzoo['ra'], cat_galaxyzoo['dec'], unit=(u.deg,u.deg))
for i in range(len(coo)):
    print 'Processing galaxy ', i, ' - Coo: ', coo[i]
    result = SDSS.query_region(coo[i], spectro=True, photoobj_fields=['ra','dec','u','g','r','i','z'], radius=2*u.arcsec)
    if i==0:
        sdss_data=result
    else:
        sdss_data.add_row(np.array(result)[0])

cat_sdss_espirales=pd.DataFrame(np.array(sdss_data))
cat_sdss_espirales

## Apendices

Descargar el catalogo completo de Galaxy zoo

In [ ]:
import requests, zipfile, StringIO

In [ ]:
cat_url='http://galaxy-zoo-1.s3.amazonaws.com/GalaxyZoo1_DR_table2.csv.zip'

r = requests.get(cat_url)
z = zipfile.ZipFile(StringIO.StringIO(r.content))

cat_file='GalaxyZoo1_DR_table2.csv'
cat_data=pd.read_csv(z.open(cat_file))
cat_data.head()

In [ ]:
cat_data=cat_data.sort_values('P_EL_DEBIASED',ascending=0)
cat_elliptical=cat_data[cat_data['ELLIPTICAL']==1][0:100]
cat_elliptical['ra']=Angle(cat_elliptical['RA'], unit=u.hourangle).degree
cat_elliptical['dec']=Angle(cat_elliptical['DEC'], unit=u.degree).degree
cat_elliptical=cat_elliptical.rename(columns = {'OBJID':'ID','NVOTE':'votos'})
cat_elliptical=cat_elliptical[['ID','ra','dec','votos']].reset_index(drop=True)

cat_elliptical.to_csv('../data/SDSS_galaxyzoo_elipticas.csv',index=False)

In [ ]:
cat_data=cat_data.sort_values('P_CS_DEBIASED',ascending=0)
cat_spiral=cat_data[cat_data['SPIRAL']==1][0:100]
cat_spiral['ra']=Angle(cat_spiral['RA'], unit=u.hourangle).degree
cat_spiral['dec']=Angle(cat_spiral['DEC'], unit=u.degree).degree
cat_spiral=cat_spiral.rename(columns = {'OBJID':'ID','NVOTE':'votos'})
cat_spiral=cat_spiral[['ID','ra','dec','votos']].reset_index(drop=True)

cat_spiral.to_csv('../data/SDSS_galaxyzoo_espirales.csv',index=False)